# Provisioning

In [ ]:
%%writefile MAIA-FL/Provisioning/Demo_project.yaml
api_version: 3
name: demo
description: NVIDIA FLARE Demo project

participants:
  - name: maia
    type: server
    org: maia
    fed_learn_port: 30023
    admin_port: 30052
  - name: maia-client-1
    org: maia
    type: client
    components:
      resource_manager:    # This id is reserved by system.  Do not change it.
        path: nvflare.app_common.resource_managers.gpu_resource_manager.GPUResourceManager
        args:
          num_of_gpus: 1
          mem_per_gpu_in_GiB: 11
      resource_consumer:    # This id is reserved by system.  Do not change it.
        path: nvflare.app_common.resource_consumers.gpu_resource_consumer.GPUResourceConsumer
        args:

# The same methods in all builders are called in their order defined in builders section
builders:
  - path: nvflare.lighter.impl.workspace.WorkspaceBuilder
    args:
      template_file: master_template.yml
  - path: nvflare.lighter.impl.template.TemplateBuilder
  - path: docker_compose.DockerBuilder
    args:
      base_image: maiacloud/nvflare:1.3
      requirements_file: docker_compose_requirements.txt
  - path: helm_chart.HelmChartBuilder
    args:
      docker_image: maiacloud/nvflare:1.3
      storage_class: nfs-client
      sftp_server_config:
        image: registry.maia-cloud.com/maia/maia-filebrowser:1.0
        env:
          RUN_FILEBROWSER: "True"
          user: "admin"
          password: "admin"
          n_users: "1"
        mount_path: /home/nvflare
  - path: nvflare.lighter.impl.static_file.StaticFileBuilder
    args:
      # config_folder can be set to inform NVIDIA FLARE where to get configuration
      config_folder: config

      # scheme for communication driver (currently supporting the default, grpc, only).
      # scheme: grpc

      # app_validator is used to verify if uploaded app has proper structures
      # if not set, no app_validator is included in fed_server.json
      # app_validator: PATH_TO_YOUR_OWN_APP_VALIDATOR

      # when docker_image is set to a docker image name, docker.sh will be generated on server/client/admin
      # docker_image:

      # download_job_url is set to http://download.server.com/ as default in fed_server.json.  You can override this
      # to different url.
      # download_job_url: http://download.server.com/

      overseer_agent:
        path: nvflare.ha.dummy_overseer_agent.DummyOverseerAgent
        # if overseer_exists is true, args here are ignored.  Provisioning
        #   tool will fill role, name and other local parameters automatically.
        # if overseer_exists is false, args in this section will be used and the sp_end_point
        # must match the server defined above in the format of SERVER_NAME:FL_PORT:ADMIN_PORT
        # 
        overseer_exists: false
        args:
          sp_end_point: maia-small.cloud.cbh.kth.se:30023:30052

  - path: nvflare.lighter.impl.cert.CertBuilder
  - path: nvflare.lighter.impl.signature.SignatureBuilder
  - path: distribution.DistributionBuilder
    args:
      zip_password: True
